In [ ]:
import spacy
import pandas as pd
import numpy as np
import nltk 
import string
import os
from spacy.lang.en.stop_words import STOP_WORDS
from tqdm import tqdm
nlp = spacy.load('en_core_web_md')

In [ ]:
adele = pd.read_csv("/Volumes/Macintosh HD – dane/GitHub/masters/first semester/Unsupervised Learning/article_2/data/adele.txt", sep="\b", names=['column1'], header=None)

In [ ]:
folder_path = "/Volumes/Macintosh HD – dane/GitHub/masters/first semester/Unsupervised Learning/article_2/data"
os.chdir(folder_path)

In [ ]:
def read_text_file(file_path):
    with open(file_path, 'r') as f:
        lyrics = f.read()
    return lyrics

In [157]:
def get_lyrics(list_of_songs = []):
    
    for file in os.listdir():

        if file.endswith(".txt"):
            file_path = f"{folder_path}/{file}"

            lyrics = pd.read_csv(file_path, sep='\b', quoting=3, encoding='utf-8', header=None, names=['lines'])
            lyrics_list = lyrics['lines'].tolist()

            list_of_songs.append(lyrics_list)
    return list_of_songs

In [ ]:
def lyrics_preprocess(songs_list, stopwords, songs_vectors = [], stop_words_check = []):

    for song in tqdm(songs_list):

        text = " ".join(song)
        doc = nlp(text)

        tokens  = [token.text for token in doc] # tokenize
        tokens = [token.lemma_ for token in doc] # lemmantize

        tokens = [token for token in tokens if token not in string.punctuation] # remove punctuation
        tokens = [token.lower() for token in tokens] # lower words
        tokens = [item for item in tokens if item not in stopwords] # remove stopwords

        for word in tokens:
            if word in stopwords:
                stop_words_check.append(word)
            assert len(stop_words_check) == 0, 'Error: not all of the stopwords were deleted from text'
        
        tokens_concat = " ".join(tokens)
        sentence_vec = nlp(tokens_concat)
        songs_vectors.append(sentence_vec.vector)
        
    return songs_vectors
        




In [151]:
def create_column_names(list_of_columns = []):

    for file in os.listdir():

        
        file = file[:-4]
        list_of_columns.append(file)

        if '.DS_S' in list_of_columns:
            list_of_columns.remove('.DS_S')

    return list_of_columns

In [ ]:
stop = STOP_WORDS
stop.update(['...', '....', '1', '2', '3', '4', '5', 'chorus', ':]', '[:'])

In [ ]:
lyrics = get_lyrics()
lyrics_vectors = lyrics_preprocess(test1, stopwords = stop)
df_columns = create_column_names()

In [158]:
songs_df = pd.DataFrame(test2).T
songs_df.columns = df_columns

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,195
0,0.488899,-0.151497,0.281931,0.866229,0.332050,0.464633,-0.052686,0.628423,1.015842,0.393763,...,0.799079,0.404629,1.180542,0.407606,0.457071,0.121589,0.254363,0.492601,0.342824,1.013985
1,1.610341,0.748771,1.261264,1.241061,1.255925,1.078212,1.197751,1.076042,0.849392,1.128111,...,1.237406,1.262244,1.398354,1.339310,1.205130,1.069831,0.998072,1.366088,1.295770,1.322053
2,-1.936098,-1.636477,-2.027798,-2.377264,-1.891242,-2.123817,-2.158009,-1.888964,-2.310341,-2.193906,...,-2.352120,-2.181932,-2.494427,-2.476290,-2.033452,-2.097285,-2.303176,-2.186932,-2.072237,-2.356970
3,-0.821185,-0.192223,-0.096713,-0.816632,-0.467427,-0.406551,-0.025675,-0.709705,-1.149756,-0.394666,...,-0.881137,-0.501777,-1.125783,-0.479958,-0.329150,-0.649858,-0.553310,-0.386956,-0.240733,-0.748238
4,-0.371704,0.994015,-0.321496,-0.149797,0.647207,0.377207,0.430123,0.280071,-0.508992,0.116053,...,-0.039799,-0.022557,-0.342595,0.280042,-0.274202,-0.136441,0.717278,0.202165,-0.023635,-0.443886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,-0.493059,-0.291954,-0.289812,-1.252576,-0.793673,-0.642873,-0.548541,-0.638050,-1.192136,-0.699784,...,-1.088211,-0.866636,-1.192166,-0.589690,-0.602087,-0.377834,-0.635343,-0.790784,-0.691271,-0.865328
296,0.522071,0.441203,0.065536,0.393009,-0.061658,0.441511,0.044495,0.520668,0.859490,0.324131,...,0.365115,0.185198,0.098396,0.172240,0.137928,0.215483,0.558434,-0.033671,0.217477,0.631482
297,1.234423,0.297694,1.209336,1.530120,0.754371,0.821359,0.996710,1.127094,1.903543,1.380986,...,1.836874,1.452400,2.122900,1.432235,1.827290,1.020568,1.159650,1.329938,1.440136,1.787147
298,-2.219106,-2.071717,-1.386191,-2.359582,-2.035150,-2.070417,-1.734162,-2.202337,-1.717645,-1.705081,...,-1.845270,-1.645657,-1.936949,-1.770404,-1.475431,-1.807916,-2.087821,-1.989501,-1.534161,-1.685631
